## Alice in Wonderland TMDs

K, note to self here: the auto-aligner isn't aligning the full text correctly (at least, not for DE-ZH), so split into chapters and then align

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import glob
import os
import subprocess

import joblib
import pandas as pd
import numpy as np

import trglobals

In [16]:
os.getcwd()

'C:\\Dropbox\\Translation\\code00_tmd'

In [17]:
trglobals.init_embedding_model()

No model available for am
de model already initialized
el model already initialized
en model already initialized
it model already initialized
No model available for kk
ru model already initialized
vi model already initialized
zh model already initialized


In [18]:
alice_path = "../Texts/Alice/"
de_path = os.path.join(alice_path, "alice_de")
de_cleaned_path = os.path.join(alice_path, "alice_de_cleaned")
if not os.path.isdir(de_cleaned_path):
    os.mkdir(de_cleaned_path)

In [19]:
def read_text(fpath):
    with open(fpath, 'r', encoding='utf-8') as infile:
        text = infile.read()
    return text

In [20]:
de_fpaths = glob.glob(os.path.join(de_path, "*.txt"))

In [21]:
def custom_clean(text, lang):
    if lang == "de":
        text_clean = text.replace("_","").replace("\n"," ").strip()
    elif lang == "el":
        text_clean = text.strip()
    elif lang == "en":
        text_clean = text.replace("_","").replace("*","").strip()
    elif lang == "ru":
        text_clean = text.strip()
    elif lang == "vi":
        text_clean = text.strip()
    elif lang == "zh":
        text_clean = text.replace("\u3000"," ").strip()
    else:
        print(f"Warning: custom_clean() called with an invalid lang: {lang}")
        text_clean = text
    return text_clean


In [22]:
for cur_de_fpath in de_fpaths:
    cur_de_fname = os.path.basename(cur_de_fpath)
    de_text = read_text(cur_de_fpath)
    de_text_clean = custom_clean(de_text, "de")
    de_clean_fpath = os.path.join(de_cleaned_path, cur_de_fname)


In [23]:
lang2 = "el"

In [24]:
l2_path = os.path.join(alice_path, f"alice_{lang2}")
l2_cleaned_path = os.path.join(alice_path, f"alice_{lang2}_cleaned")
if not os.path.isdir(l2_cleaned_path):
    os.mkdir(l2_cleaned_path)
l2_cleaned_path

'../Texts/Alice/alice_el_cleaned'

In [25]:
l2_fpaths = glob.glob(os.path.join(l2_path, "*.txt"))
l2_fpaths

['../Texts/Alice/alice_el\\ch01.alice_el.txt',
 '../Texts/Alice/alice_el\\ch02.alice_el.txt',
 '../Texts/Alice/alice_el\\ch03.alice_el.txt',
 '../Texts/Alice/alice_el\\ch04.alice_el.txt',
 '../Texts/Alice/alice_el\\ch05.alice_el.txt',
 '../Texts/Alice/alice_el\\ch06.alice_el.txt',
 '../Texts/Alice/alice_el\\ch07.alice_el.txt',
 '../Texts/Alice/alice_el\\ch08.alice_el.txt',
 '../Texts/Alice/alice_el\\ch09.alice_el.txt',
 '../Texts/Alice/alice_el\\ch10.alice_el.txt',
 '../Texts/Alice/alice_el\\ch11.alice_el.txt',
 '../Texts/Alice/alice_el\\ch12.alice_el.txt']

In [26]:
for cur_l2_fpath in l2_fpaths:
    cur_l2_fname = os.path.basename(cur_l2_fpath)
    l2_text = read_text(cur_l2_fpath)
    l2_text_clean = custom_clean(l2_text, lang2)
    l2_clean_fpath = os.path.join(l2_cleaned_path, cur_l2_fname)
    with open(l2_clean_fpath, 'w', encoding='utf-8') as outfile:
        outfile.write(l2_text_clean)

In [27]:
l2_text_clean

'Κεφάλαιο 12\n\nΗ κατάθεση της Αλίκης\n\n- Παρούσα! φώναξε η Αλίκη, ξεχνώντας ολότελα, μέσα στην\nταραχή της, πόσο πολύ είχε ψηλώσει τα τελευταία λίγα λεπτά\nκαι πετάχτηκε όρθια τόσο βιαστικά, που αναποδογύρισε το έδρανο των ενόρκων με την άκρη της φούστας της, γκρεμίζοντας\nόλους τους ενόρκους πάνω στα κεφάλια του κόσμου, που καθόταν από κάτω. Όπως τους είδε σκορπισμένους εδώ κι εκεί, της\n\nθύμισαν έντονα μια γυάλα με χρυσόψαρα που είχε αναποδογυρίσει κατά λάθος την προηγούμενη βδομάδα.\n\n- Αχ, σας ζητώ συγνώμη! φώναξε φοβερά συγχυσμένη κι\nάρχισε να μαζεύει από χάμω τους ενόρκους όσο πιο γρήγορα\nμπορούσε, γιατί το ατύχημα με τα χρυσόψαρα δεν έλεγε να\nφύγει από το μυαλό της κι είχε μιαν αόριστη εντύπωση πως\nέπρεπε να τους μαζέψει αμέσως και να τους ξαναβάλει στο\nέδρανο, αλλιώς θα πέθαιναν.\n\n- Η δίκη δεν μπορεί να συνεχιστεί, είπε ο Βασιλιάς με πολύ\nσοβαρή φωνή, μέχρι να επιστρέφουν όλοι οι ένορκοι στις κανονικές τους θέσεις. Είπα όλοι! επανέλαβε με πολλή έμφαση,\nκοιτάζοντας 

### Align

In [28]:
aligned_path = "../Texts_Aligned/Alice/"
if not os.path.isdir(aligned_path):
    os.mkdir(aligned_path)
cur_aligned_path = os.path.join(aligned_path, f"de_alice.{lang2}_alice")
if not os.path.isdir(cur_aligned_path):
    os.mkdir(cur_aligned_path)
cur_aligned_path

'../Texts_Aligned/Alice/de_alice.el_alice'

Load the cleaned texts for each lang

In [29]:
de_cleaned_fpaths = glob.glob(os.path.join(de_cleaned_path, "*.txt"))
l2_cleaned_fpaths = glob.glob(os.path.join(l2_cleaned_path, "*.txt"))

In [30]:
def run_align(lf_cmd):
    orig_dir = os.getcwd()
    os.chdir(trglobals.lf_path)
    #print(f"Running command from {os.getcwd()}")
    print(f"Running command: {lf_cmd}")
    result = subprocess.check_output(lf_cmd, shell=True)
    #align_logs.append(result)
    os.chdir(orig_dir)
    return result

In [32]:
align_logs = {}
for de_fpath, l2_fpath in zip(de_cleaned_fpaths, l2_cleaned_fpaths):
    de_fname = os.path.basename(de_fpath)
    ch_num_str = de_fname.split(".")[0]
    print(f"Processing {ch_num_str}")
    cur_aligned_fpath = os.path.join(cur_aligned_path, f"{ch_num_str}.tsv")
    lf_cmd = trglobals.get_lf_cmd('de', lang2, de_fpath, l2_fpath, cur_aligned_fpath)
    try:
        result = run_align(lf_cmd)
        align_logs[ch_num_str] = result
    except subprocess.CalledProcessError as e:
        print(e)
        align_logs[ch_num_str] = e
        break

Processing ch01
Running command: LF_aligner_4.1.exe --filetype="t" --infiles="../Texts/Alice/alice_de_cleaned\ch01.alice_de.txt","../Texts/Alice/alice_el_cleaned\ch01.alice_el.txt" --outfile="../Texts_Aligned/Alice/de_alice.el_alice\ch01.tsv" --languages="de","el" --segment="y" --review="n" --tmx="n"
Processing ch02
Running command: LF_aligner_4.1.exe --filetype="t" --infiles="../Texts/Alice/alice_de_cleaned\ch02.alice_de.txt","../Texts/Alice/alice_el_cleaned\ch02.alice_el.txt" --outfile="../Texts_Aligned/Alice/de_alice.el_alice\ch02.tsv" --languages="de","el" --segment="y" --review="n" --tmx="n"
Processing ch03
Running command: LF_aligner_4.1.exe --filetype="t" --infiles="../Texts/Alice/alice_de_cleaned\ch03.alice_de.txt","../Texts/Alice/alice_el_cleaned\ch03.alice_el.txt" --outfile="../Texts_Aligned/Alice/de_alice.el_alice\ch03.tsv" --languages="de","el" --segment="y" --review="n" --tmx="n"
Processing ch04
Running command: LF_aligner_4.1.exe --filetype="t" --infiles="../Texts/Alice/a

### TMD on output

In [33]:
# Collect all the chapters into a single df

In [34]:
aligned_fpaths = glob.glob(os.path.join(cur_aligned_path, "*.tsv"))
aligned_fpaths

['../Texts_Aligned/Alice/de_alice.el_alice\\ch01.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch02.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch03.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch04.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch05.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch06.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch07.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch08.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch09.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch10.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch11.tsv',
 '../Texts_Aligned/Alice/de_alice.el_alice\\ch12.tsv']

In [35]:
all_dfs = []
for cur_aligned_fpath in aligned_fpaths:
    cur_fname = os.path.basename(cur_aligned_fpath)
    print(f"Computing tmd for {cur_fname}")
    cur_align_df = pd.read_csv(cur_aligned_fpath, delimiter="\t", header=None)
    cur_align_df.columns = ['de',lang2,'alignment_id']
    cur_align_df['tmd'] = trglobals.compute_tmd_df(cur_align_df)
    all_dfs.append(cur_align_df)

Computing tmd for ch01.tsv


100%|██████████| 534/534 [00:00<00:00, 584.49it/s]


Computing tmd for ch02.tsv


100%|██████████| 522/522 [00:01<00:00, 502.46it/s]


Computing tmd for ch03.tsv


...

100%|██████████| 330/330 [00:00<00:00, 507.46it/s]


Computing tmd for ch11.tsv


100%|██████████| 296/296 [00:00<00:00, 478.08it/s]


Computing tmd for ch12.tsv


100%|██████████| 310/310 [00:00<00:00, 439.61it/s]


In [36]:
full_df = pd.concat(all_dfs)

In [37]:
l2_mean = full_df[~np.isinf(full_df['tmd'])]['tmd'].mean()
l2_mean

1.0689026715169005

In [41]:
joblib.dump(l2_mean, f'./lang_stats/alice_{lang2}_mean.pkl')

['./lang_stats/alice_el_mean.pkl']